# **GATEWAY MAIN PROGRAM FOR IOT LAB**
*Author: Minh Nguyen*  
*Date: October 30th, 2021*

## **Overview**
___
The bulk of this python program is to running a simple gateway that is capable of receiving data from the [Adafruit server](https://io.adafruit.com/ "go to Adafruit main page") (broker) and dispatching it to an IoT Arduino node. The gateway also monitors the data sent from the Arduino and publishes it to the server, therefore establishing a form of communication between the broker and the sensor.

The attributes of this simple implementation of gateway are:
- The gateway receives **temperature** and **humidity** data from the Arduino and publishes them to the broker.
- The gateway listens to **led** and **servo** data from the broker and dispatches them to the Arduino.
- The communication protocol between the gateway and the broker is **MQTT**, which is implemented through the use of the [paho-mqtt](https://pypi.org/project/paho-mqtt/ "go to paho-mqtt pydoc") library.
- The communication protocol between the gateway and the Arduino is **serial UART**.

## **Implementation**
___
I will walk you through the detail implementation of this python program, including the idea and the approach I used to establish a stable communication between the gateway and the broker.

 ### **The keep-in-mind tactic, recommended by Adafruit themselves.**
The Adafruit broker has its own [API](https://pypi.org/project/adafruit-io/) for the end-users. This API is optimized and provides many ease-of-use interfaces. Because we are not going to use the Adafruit API, we need to comply with the best practices recommended by the broker. The quote below is cited from the Adafruit IO MQTT api [documentation](https://io.adafruit.com/api/docs/mqtt.html#adafruit-io-mqtt-api), under the section entitled *Writing Safe MQTT Code*:

> **Always. Use. Delay.**  
> Make sure there is time in between messages to the Adafruit IO MQTT broker. If you're saving data in an Arduino or CircuitPython sketch inside the program's main loop, make sure you're delaying, sleeping, or using some other technique to put space in between messages.

During implementation, I reckoned that there are many routines of the MQTT protocol that run on seperate threads. For example the callback to `on_message()` or `on_disconnect()` function is invoked on its own thread, without any interfering to or impacted from the big `while` of the gateway. This is kinda out of control because according to the safe guide from Adafruit, we must have someway to control the delay between incoming messages. In this situation, because we have no way to tell when the message will come, there are high chances the broker will send out a lot of data to the gateway. If the `on_message()` callback were to handle all data and dispatch to the Arduino, this would lead to serious problem as instructed by the Adafruit documentation. To address this, I use a **shared-buffer queue** for the incoming messages. The `on_message()` callback only bother putting the incoming data onto the queue, and our big `while` loop will constantly check if the queue is empty or not. This way, we can put our control to the `while` loop, simply by inserting a `time.delay()` into it, and the Arduino will happily received data from the dispatching of the while loop without fearing of data lost. The pseudocode is as follow:

```python
messageBuffer = queue.Queue()

def on_message(data, ...):
    messageBuffer.put(data)

...

while True:
    if not messageBuffer.empty():
        data = messageBuffer.get()
        dispatch(data)
        
    time.delay(SCAN_DELAY)
```

### **The lock-thread behavior of routines.**
The asynchronous behavior of callbacks not only makes the control of incoming messages harder, but also put the data race problems at high risk. There are 2 main concerns regarding this issue:  
- the order of debuggings message on terminal
- the use of `userdata` object

Consider this piece of code in the program:  

```python
gtw = mqtt.Client(...)  # we instantiate an MQTT object

...

gtw.connect(...)    # establish a connection to the broker
                    # an asynchronus callback will be invoked on successful connection
gtw.subscribe(...)  # issue several subscriptions to topics on the server
                    # asynchronus callbacks will also invoked
```

Because we are printing every messages of the callbacks to the terminal, it would be a mess if we leave the asynchronous routines out of control. Furthermore, if we can ensure the callbacks get invoked in order, then later on we can easily add error control to the program.

### **Gateway program**
The first step is to import necessary packages for our program.

In [ ]:
import paho.mqtt.client as mqtt
import time, queue, serial, threading
from serial.serialutil import SerialException
import serial.tools.list_ports as serialtool

Because we are going to need several hard-coded texts, we should declare some symbolic constants.

In [ ]:
# Broker Configurations
HOST_NAME = "io.adafruit.com"   # using Adafruit IO server
HOST_PORT = 8883                # secure connection
USERNAME  = "***"
PASSWORD  = "***"

# Gateway Configurations
GATEWAY_ID  = "GTW002"
GROUP_KEY   = "iot-lab"
PUB_QOS = 1
SUB_QOS = 1
STAT_TOPIC  = "gateway-status"
SUBSCRIBE_TOPICS = [
    "arduino-led",      # record led status
    "arduino-servo",    # record change in servo angle
    STAT_TOPIC          # listen to disconnect signal from the broker
]
SCAN_DELAY  = 1.0
TIME_OUT    = 5.0       # maximum waiting time for blocking
MAX_FAILED_ATTEMPTS = 3

# Devices Configurations
BAUDRATE    = 9600    # Arduino default baudrate
COM_TOKEN   = 'USB Serial Device'

To make the debugging console more descriptive, let add a color scheme.

In [ ]:
class col:
    cdtag   = '\033[35;1m'
    mestag  = '\033[1;34m'
    pubtag  = '\033[31;1m'
    subtag  = '\033[0;33m'
    good    = '\033[92m'
    bad     = '\033[31m'
    user    = '\033[0;95m'
    topic   = '\033[1;37m'
    message = '\033[0;94m'
    stage   = '\033[0;93m'
    esc     = '\033[0m'

The next step is to settle down several global variables to be used in our program. The majority of the varibales is for thread controling, while some others are used accross the functions. 

In [ ]:
block       = False # flag to block thread
exquit      = False # flag to raise disconnect interrupt
greeting    = True  # flag to indicate whether this is the first time we connect
reinit      = False # flag to call init() procedure on reconnect
texceed     = False # flag to indicate we've reached maximum blocking time
failcount   = 0     # count attempts on fail activity
msg         = ''    # hold buffer message read from serial

messageBuffer = queue.Queue()   # holds incoming messages
devicesBuffer = queue.Queue()   # holds outgoing messages
terminate     = False   # control thread termination
scanning      = False   # flag to indicate initial setup

Now it's time to define some important callbacks for our MQTT protocol. The first callback is the `on_connect()` method, which will be invoked after any call to the `connect()` method of the gateway object.

In [ ]:
def on_connect(client, userdata, flags, rc):
    if rc != 0:
        print(f"{col.cdtag}[C]{col.esc} {GATEWAY_ID} failed to connect to {USERNAME}: {col.bad}{mqtt.connack_string(rc)}{col.esc}")
    else:
        global greeting
        if greeting:
            print(f"{col.cdtag}[C]{col.esc} {GATEWAY_ID} connected to {col.user}{USERNAME}{col.esc} with result: {col.good}{mqtt.connack_string(rc)}{col.esc}")
            greeting = False
            global block
            block = False
        else:
            print(f"{col.cdtag}[C]{col.esc} Reconnected to {col.user}{USERNAME}{col.esc} with result: {col.good}{mqtt.connack_string(rc)}{col.esc}")